In [1]:
import pandas as pd
import numpy as np
from core.match import Match
from core.payoff import PayoffMatrix
from strategies.basic import AlwaysCooperate, AlwaysDefect
from strategies.advanced import TitForTat, Random
from tournaments.roundrobin import RoundRobinTournament

### Run di esempio dello scheletro
#### Creazione delle istanze necessarie per definire i parametri del gioco

Creiamo una Matrice di Payoff usando il costruttore di default della classe e creiamo due giocatori.
I due giocatori sono istanze separate della classe strategy per cui tramite Polimorfismo ereditano la struttura della classe ma differiscono solamente per algoritmo di decisione.

In [9]:
M = PayoffMatrix(R=1, S=0, T=5, P=1)

player1 = AlwaysCooperate(name = 'Pippo')
player2 = AlwaysDefect(name = 'Topolino')

ValueError: Vincolo violato: deve valere T > R > P > S. Ricevuto: T=5, R=1, P=1, S=0

Adesso che abbiamo i parametri necessari per definire un match creiamo l'orchestratore della struttura:  Match.
Crea un istanza con i dati necessari per runnare tutto l'algoritmo. 

In [3]:
m = Match(player1,player2,M)

results = m.run(rounds=10)

Adesso effettuiamo il display dei risultati

In [4]:
df = pd.DataFrame({
    f"{results['agent_a']} (Move)": results['history_a'], 
    f"{results['agent_b']} (Move)": results['history_b'],
    f"{results['agent_a']} (Payoff)": results['payoffs_a'],
    f"{results['agent_b']} (Payoff)": results['payoffs_b'],
    f"{results['agent_a']} (Cum. Score)": results['cumulative_scores_a'],
    f"{results['agent_b']} (Cum. Score)": results['cumulative_scores_b']
})
display(df.head())

,Pippo (Move),Topolino (Move),Pippo (Payoff),Topolino (Payoff),Pippo (Cum. Score),Topolino (Cum. Score)
0,C,D,0,5,0,5
1,C,D,0,5,0,10
2,C,D,0,5,0,15
3,C,D,0,5,0,20
4,C,D,0,5,0,25


Adesso vediamo come creare un altro match. Da tenere a mente che la classe strategy non tiene conto della storia degli eventi in match precedenti quindi ogni partita ricomincia da capo. La memoria non è necessaria.

In [5]:
player3 = TitForTat(name='Pluto')

m2= Match(player2,player3,M)
results2 = m2.run(rounds = 100)
print(f"Risultato Finale:")
print(f"{results2['agent_a']}: {results2['score_a']} punti")
print(f"{results2['agent_b']}: {results2['score_b']} punti")
# Se vuoi vedere il dettaglio round per round:
df2 = pd.DataFrame({
    f"{results2['agent_a']} (Move)": results2['history_a'], 
    f"{results2['agent_b']} (Move)": results2['history_b'],
    f"{results2['agent_a']} (Payoff)": results2['payoffs_a'],
    f"{results2['agent_b']} (Payoff)": results2['payoffs_b'],
    f"{results2['agent_a']} (Cum. Score)": results2['cumulative_scores_a'],
    f"{results2['agent_b']} (Cum. Score)": results2['cumulative_scores_b']
})
display(df2.head())



Risultato Finale:
Topolino: 104 punti
Pluto: 99 punti


,Topolino (Move),Pluto (Move),Topolino (Payoff),Pluto (Payoff),Topolino (Cum. Score),Pluto (Cum. Score)
0,D,C,5,0,5,0
1,D,D,1,1,6,1
2,D,D,1,1,7,2
3,D,D,1,1,8,3
4,D,D,1,1,9,4


### Esempio di Modularità per lo sviluppo in parallelo
Adesso voglio usare un algoritmo per decidere che richiede dei parametri personalizzati. Non devo cambiare assolutamente nulla in Match, Tournament e Core in quanto il polimorfismo mi permette di creare classi custom a partire dal template con costruttori personalizzati. 

In [6]:
player4 = Random(name='Ping Pong Un') # Default random player with p = 0.5
player5 = Random(p=0.7,name='Topo Gigio')

m3= Match(player4,player5,M)
results3 = m3.run(rounds = 30)
print(f"Risultato Finale:")
print(f"{results3['agent_a']}: {results3['score_a']} punti")
print(f"{results3['agent_b']}: {results3['score_b']} punti")
# Se vuoi vedere il dettaglio round per round:
df3 = pd.DataFrame({
    f"{results3['agent_a']} (Move)": results3['history_a'], 
    f"{results3['agent_b']} (Move)": results3['history_b'],
    f"{results3['agent_a']} (Payoff)": results3['payoffs_a'],
    f"{results3['agent_b']} (Payoff)": results3['payoffs_b'],
    f"{results3['agent_a']} (Cum. Score)": results3['cumulative_scores_a'],
    f"{results3['agent_b']} (Cum. Score)": results3['cumulative_scores_b']
})
display(df3.head())

Risultato Finale:
Ping Pong Un: 69 punti
Topo Gigio: 69 punti


,Ping Pong Un (Move),Topo Gigio (Move),Ping Pong Un (Payoff),Topo Gigio (Payoff),Ping Pong Un (Cum. Score),Topo Gigio (Cum. Score)
0,C,D,0,5,0,5
1,D,D,1,1,1,6
2,D,D,1,1,2,7
3,C,D,0,5,2,12
4,C,D,0,5,2,17


### Using tournament class to create a Round Robin tournament

First create all the attributes needed: a list of players, the number of round for each match between two players and a Payoff Matrix.
Then use the method play_tournament to dispute al matches.

In [7]:
# Matches Parameters
M = PayoffMatrix()
players = [AlwaysCooperate(), AlwaysDefect(), TitForTat(), Random(name = 'Mainly Nice', p=0.7), Random(name='Mainly Bad', p=0.3)]
N_rounds = 50

tournament = RoundRobinTournament(players, N_rounds, M)
tournament.play_tournament(output = False) # Disabling output for debugging

Now let's see how to access results for analysis. The whole event log is stored inside the class as an attribute.
To address logical clarity here an explicit use of the attributes:

In [8]:
N_players = tournament.N

# Winninng statistics for each player. Analyze to create leaderboards
N_wins = tournament.N_wins
N_ties = tournament.N_ties
Total_score = tournament.Total_score

# Detailed log for every match
results = tournament.results # List of dictionaries with results for each match
dfs = tournament.dfs

# Displaying Leaderboard

print('Player name , Wins , Ties , Total Score')
for i in range(N_players):
    print(players[i].name,' ' ,N_wins[i],'', N_ties[i] , ' ' ,Total_score[i])

# Note thar dfs are created from all the attributes stored in results. This is just for convenience.
for df in dfs:
    display(df.head())

Player name , Wins , Ties , Total Score
AlwaysCooperate   0.0  1.0   282.0
AlwaysDefect   4.0  0.0   600.0
TitForTat   0.0  2.0   432.0
Mainly Nice   1.0  1.0   369.0
Mainly Bad   3.0  0.0   553.0


,AlwaysCooperate (Move),AlwaysDefect (Move),AlwaysCooperate (Payoff),AlwaysDefect (Payoff),AlwaysCooperate (Cum. Score),AlwaysDefect (Cum. Score)
0,C,D,0,5,0,5
1,C,D,0,5,0,10
2,C,D,0,5,0,15
3,C,D,0,5,0,20
4,C,D,0,5,0,25


,AlwaysCooperate (Move),TitForTat (Move),AlwaysCooperate (Payoff),TitForTat (Payoff),AlwaysCooperate (Cum. Score),TitForTat (Cum. Score)
0,C,C,3,3,3,3
1,C,C,3,3,6,6
2,C,C,3,3,9,9
3,C,C,3,3,12,12
4,C,C,3,3,15,15


,AlwaysCooperate (Move),Mainly Nice (Move),AlwaysCooperate (Payoff),Mainly Nice (Payoff),AlwaysCooperate (Cum. Score),Mainly Nice (Cum. Score)
0,C,C,3,3,3,3
1,C,C,3,3,6,6
2,C,D,0,5,6,11
3,C,C,3,3,9,14
4,C,C,3,3,12,17


,AlwaysCooperate (Move),Mainly Bad (Move),AlwaysCooperate (Payoff),Mainly Bad (Payoff),AlwaysCooperate (Cum. Score),Mainly Bad (Cum. Score)
0,C,D,0,5,0,5
1,C,C,3,3,3,8
2,C,D,0,5,3,13
3,C,C,3,3,6,16
4,C,D,0,5,6,21


...

,AlwaysDefect (Move),Mainly Nice (Move),AlwaysDefect (Payoff),Mainly Nice (Payoff),AlwaysDefect (Cum. Score),Mainly Nice (Cum. Score)
0,D,C,5,0,5,0
1,D,C,5,0,10,0
2,D,D,1,1,11,1
3,D,D,1,1,12,2
4,D,D,1,1,13,3


,AlwaysDefect (Move),Mainly Bad (Move),AlwaysDefect (Payoff),Mainly Bad (Payoff),AlwaysDefect (Cum. Score),Mainly Bad (Cum. Score)
0,D,D,1,1,1,1
1,D,C,5,0,6,1
2,D,C,5,0,11,1
3,D,C,5,0,16,1
4,D,D,1,1,17,2


,TitForTat (Move),Mainly Nice (Move),TitForTat (Payoff),Mainly Nice (Payoff),TitForTat (Cum. Score),Mainly Nice (Cum. Score)
0,C,C,3,3,3,3
1,C,C,3,3,6,6
2,C,D,0,5,6,11
3,D,C,5,0,11,11
4,C,C,3,3,14,14


,TitForTat (Move),Mainly Bad (Move),TitForTat (Payoff),Mainly Bad (Payoff),TitForTat (Cum. Score),Mainly Bad (Cum. Score)
0,C,C,3,3,3,3
1,C,D,0,5,3,8
2,D,D,1,1,4,9
3,D,C,5,0,9,9
4,C,D,0,5,9,14


,Mainly Nice (Move),Mainly Bad (Move),Mainly Nice (Payoff),Mainly Bad (Payoff),Mainly Nice (Cum. Score),Mainly Bad (Cum. Score)
0,C,D,0,5,0,5
1,D,D,1,1,1,6
2,C,D,0,5,1,11
3,D,D,1,1,2,12
4,C,D,0,5,2,17
